<a href="https://colab.research.google.com/github/suubkiim/Pytorch-practice/blob/master/RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

* RNN vs LSTM
![대체 텍스트](https://jaydottechdotblog.files.wordpress.com/2017/01/lrcns-rnn-lstm-diagram.png)

> http://blog.naver.com/PostView.nhn?blogId=koys007&logNo=221528966460&? parentCategoryNo=&categoryNo=64&viewDate=&isShowPopularPosts=true&from=search

* RNN 이해 _ 블록 설명

> https://tykimos.github.io/2017/04/09/RNN_Getting_Started/

In [0]:
import torch 
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms


# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyper-parameters
sequence_length = 28 #  MNIST 데이터에서는 행 크기 28 
input_size = 28 # MNIST 데이터에서는 열 1개 크기 28X1 
hidden_size = 128 # 28X1이 들어가서 128X1이 됨
num_layers = 2
num_classes = 10
batch_size = 100
num_epochs = 2
learning_rate = 0.01
#input size는 몇개의 input을 한번에 받을 것인지, 
#hidden size는 몇개의 output을 내보낼 것인지를 의미합니다.

# MNIST dataset
train_dataset = torchvision.datasets.MNIST(root='../../data/',
                                           train=True, 
                                           transform=transforms.ToTensor(),
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='../../data/',
                                          train=False, 
                                          transform=transforms.ToTensor())

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size, 
                                          shuffle=False)

# Recurrent neural network (many-to-one)
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)
    
    def forward(self, x):
        # Set initial hidden and cell states 
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device) 
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device) # 2x 100x 128 = #layer x #batch x #hidden size
        
        # Forward propagate LSTM
        out, _ = self.lstm(x, (h0, c0))  # out: tensor of shape (batch_size, seq_length, hidden_size) 
        
        # (h0, c0) 문법 설명 : https://pytorch.org/docs/stable/nn.html
        # c 는 cell state , 총 128 개 --> sequence length가 28 이므로

        # Decode the hidden state of the last time step
        out = self.fc(out[:, -1, :])    # -1에 해당하는 부분만 남음 100 *128 이 남음
        #100, 28, 128

        return out

model = RNN(input_size, hidden_size, num_layers, num_classes).to(device)


# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.reshape(-1, sequence_length, input_size).to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images) # 인자수 1개이므로 forward 실행
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

# Test the model
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, sequence_length, input_size).to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total)) 

# Save the model checkpoint
torch.save(model.state_dict(), 'model.ckpt')

0it [00:00, ?it/s]

9920512it [00:01, 9791965.00it/s]                            


Extracting ../../data/MNIST/raw/train-images-idx3-ubyte.gz to ../../data/MNIST/raw


  0%|          | 0/28881 [00:00<?, ?it/s]

32768it [00:00, 140862.04it/s]           
  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ../../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../../data/MNIST/raw


1654784it [00:00, 2491321.47it/s]                           
0it [00:00, ?it/s]

Extracting ../../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../../data/MNIST/raw


8192it [00:00, 51898.69it/s]            


Extracting ../../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../../data/MNIST/raw
Processing...
Done!
Epoch [1/2], Step [100/600], Loss: 0.6422
Epoch [1/2], Step [200/600], Loss: 0.4340
Epoch [1/2], Step [300/600], Loss: 0.2109
Epoch [1/2], Step [400/600], Loss: 0.2234
Epoch [1/2], Step [500/600], Loss: 0.2379
Epoch [1/2], Step [600/600], Loss: 0.2759
Epoch [2/2], Step [100/600], Loss: 0.0967
Epoch [2/2], Step [200/600], Loss: 0.1404
Epoch [2/2], Step [300/600], Loss: 0.0886
Epoch [2/2], Step [400/600], Loss: 0.1096
Epoch [2/2], Step [500/600], Loss: 0.0705
Epoch [2/2], Step [600/600], Loss: 0.0610
Test Accuracy of the model on the 10000 test images: 97.34 %


In [2]:
import torch 

out = torch.tensor([[[4,2,3],[1,5,6],[2,8,9]],[[0,2,0],[3,5,7],[9,8,10]]])
print('original\n',out)
print(out.size())
print(out[-1, :,:])
print(out[-1, :,:].size())
print(out[:, -1, :])
print(out[:, -1, :].size())
print(out[:, :,-2])
print(out[:, :,-2].size())

original
 tensor([[[ 4,  2,  3],
         [ 1,  5,  6],
         [ 2,  8,  9]],

        [[ 0,  2,  0],
         [ 3,  5,  7],
         [ 9,  8, 10]]])
torch.Size([2, 3, 3])
tensor([[ 0,  2,  0],
        [ 3,  5,  7],
        [ 9,  8, 10]])
torch.Size([3, 3])
tensor([[ 2,  8,  9],
        [ 9,  8, 10]])
torch.Size([2, 3])
tensor([[2, 5, 8],
        [2, 5, 8]])
torch.Size([2, 3])


In [0]:
# 과제2 - linear regression 부분
model2 = nn.Linear(input_size, output_size)
print(model2.state_dict()) # random하게 나옴
model2.load_state_dict(torch.load('model.ckpt'))
print(model2.state_dict()['weight']) # 저장된 값이 나옴

![LSTM 그림 설명](https://image.slidesharecdn.com/dlcvd2l6recurrentneuralnetworks-160802094750/95/deep-learning-for-computer-vision-recurrent-neural-networks-upc-2016-9-638.jpg?cb=1470131837)

![대체 텍스트](https://nyanye.com/images/rnn2.png)

![대체 텍스트](https://image.slidesharecdn.com/agistpurnndkim190430-190430140949/95/recurrent-neural-network-rnn-14-638.jpg?cb=1556635402)

![대체 텍스트](https://discuss.pytorch.org/uploads/default/original/2X/f/fb98eb0d16b722e019db59f97825aa529cb6bc08.png)

![대체 텍스트](https://image.slidesharecdn.com/automaticimagecaptiongenerator-160708051750/95/a-neural-image-caption-generator-11-638.jpg?cb=1467955312)

![대체 텍스트](https://www.mdpi.com/water/water-11-01387/article_deploy/html/images/water-11-01387-g004.png)
